# Basic CNN Tutorial

For this Jupyter notebook you will learn how to build basic CNN model and how can train your self model. 

**Requirement**
* Pytroch 1.8.1

## Import Library

In [1]:
import os
from tqdm import tqdm
import numpy as np
import torch 
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from matplotlib import pyplot as plt

## Parameters

In [2]:
model_name = "Deepfake_Model_with_CNN"
layers = 50
batch_size = 8
train_path = "./Datasets/"
output_path = "./output/"

## Preparing Data

In [3]:
# Data transforms setting    
data_transforms = {
    'train': transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
    ])
}

# Folder exist or not
if not os.path.exists(output_path):
    os.makedirs(output_path)

# cuDNN nn model optimzation
torch.backends.cudnn.benchmark= True

In [4]:
# Load training dataset
train_dataset = torchvision.datasets.ImageFolder(train_path, transform = data_transforms['train'])
# print(train_dataset.class_to_idx)

# Split training dataset
train_size = int(0.7 * len(train_dataset))
valid_size = len(train_dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])
# print("Num of Classes: " +  str(len(.classes)))
print("Training Size: " + str(train_size))
print("Training Size: " + str(valid_size))

Training Size: 8647
Training Size: 3706


In [5]:
# Creat dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= batch_size, shuffle= True, drop_last= False, num_workers= 8)
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size= batch_size, shuffle= True, drop_last= False, num_workers= 8)

print(train_loader)

## Construct The Standard CNN 

This model have 6 layers.

### Function Explaining

nn.Conv2d
torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)

**tips: stride = 1, padding = (kernel_size-1)/2 = (5-1)/2**

In [6]:
class Basic_CNN(nn.Module):
    def __init__(self):
        super(Basic_CNN, self).__init__()
        """
        Image size (256, 256)
        Input convolution (3, 256, 256)
        Conv1 output (8, 128, 128)
        Conv2 output (8, 64, 64)
        Conv3 output (16, 32, 32)
        Conv4 output (16, 8, 8)
        """
        self.conv1 = nn.Conv2d(3, 8, 3, padding= 1)
        self.conv2 = nn.Conv2d(8, 8, 5, padding= 2)
        self.conv3 = nn.Conv2d(8, 16, 5, padding= 2)
        self.conv4 = nn.Conv2d(16, 16, 5, padding= 2)

        self.act_conv = nn.ReLU(inplace= True)
        self.act_fc = nn.LeakyReLU(0.1)

        self.bn8 = nn.BatchNorm2d(8)
        self.bn16 = nn.BatchNorm2d(16)

        self.mp2 = nn.MaxPool2d(kernel_size= (2, 2))
        self.mp4 = nn.MaxPool2d(kernel_size= (4, 4))

        self.drop = nn.Dropout2d(0.5)

        self.fc1 = nn.Linear(16*8*8, 16)
        self.fc2 = nn.Linear(16, 2)

        self.log = nn.Sigmoid()
    
    def forward(self, input):
        x = self.conv1(input)
        x = self.act_conv(x)
        x = self.bn8(x)
        x = self.mp2(x)

        x = self.conv2(x)
        x = self.act_conv(x)
        x = self.bn8(x)
        x = self.mp2(x)

        x = self.conv3(x)
        x = self.act_conv(x)
        x = self.bn16(x)
        x = self.mp2(x)

        x = self.conv4(x)
        x = self.act_conv(x)
        x = self.bn16(x)
        x = self.mp4(x)

        x = x.view(x.size(0), -1)
        x = self.drop(x)
        x = self.fc1(x)
        x = self.act_fc(x)
        
        x = self.drop(x)
        x = self.fc2(x)
        x = self.log(x)
    
        return x

## Training Model

In [7]:
def train(model, n_epochs, train_loader, valid_loader, optimizer, criterion):
    train_acc_his, valid_acc_his= [],[]
    train_losses_his, valid_losses_his= [],[]
    # train_on_gpu = True
    for epoch in range(n_epochs):
        # keep track of training and validation loss
        train_loss, valid_loss = 0.0, 0.0
        train_losses, valid_losses= [],[]
        train_correct, val_correct,train_total, val_total= 0,0,0,0
        train_pred, train_target= torch.zeros(8,1), torch.zeros(8,1)
        val_pred, val_target= torch.zeros(8,1), torch.zeros(8,1)
        count= 0
        count2= 0
        print('running epoch: {}'.format(epoch + 1))
        ###################
        # train the model #
        ###################
        # model.cuda()
        model.train()
        for (data, target) in tqdm(train_loader):
            # print(target)
            # move tensors to GPU if CUDA is available
            # if train_on_gpu:
            #     data, target= data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            #calculate accuracy
            pred= output.data.max(dim= 1, keepdim= True)[1]
            train_correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            train_total += data.size(0)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_losses.append(loss.item() * data.size(0))
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            if count == 0:
                train_pred=pred
                train_target=target.data.view_as(pred)
                count= count+1
            else:
                train_pred= torch.cat((train_pred,pred), 0)
                train_target= torch.cat((train_target,target.data.view_as(pred)), 0)
        train_pred=train_pred.cpu().view(-1).numpy().tolist()
        train_target=train_target.cpu().view(-1).numpy().tolist()
        ######################    
        # validate the model #
        ######################
        model.eval()
        for (data, target) in tqdm(valid_loader):
            # move tensors to GPU if CUDA is available
            # if train_on_gpu:
            #     data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss =criterion(output, target)
            #calculate accuracy
            pred = output.data.max(dim = 1, keepdim = True)[1]
            val_correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            val_total += data.size(0)
            valid_losses.append(loss.item()*data.size(0))
            if count2==0:
                val_pred=pred
                val_target=target.data.view_as(pred)
                count2=count+1
            else:
                val_pred = torch.cat((val_pred,pred), 0)
                val_target = torch.cat((val_target,target.data.view_as(pred)), 0)
        val_pred=val_pred.cpu().view(-1).numpy().tolist()
        val_target=val_target.cpu().view(-1).numpy().tolist()
        
        # calculate average losses
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        
        # calculate average accuracy
        train_acc = train_correct/train_total
        valid_acc = val_correct/val_total
        train_acc_his.append(train_acc)
        valid_acc_his.append(valid_acc)
        train_losses_his.append(train_loss)
        valid_losses_his.append(valid_loss)
        # print training/validation statistics 
        print('\tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            train_loss, valid_loss))
        print('\tTraining Accuracy: {:.6f} \tValidation Accuracy: {:.6f}'.format(
            train_acc, valid_acc))
    return train_acc_his, valid_acc_his, train_losses_his, valid_losses_his, model

In [8]:
n_epochs= 100
model = Basic_CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, betas = (0.9, 0.999), eps = 1e-08)
train_acc_his, valid_acc_his, train_losses_his, valid_losses_his, model = train(model, n_epochs, train_loader, val_loader, optimizer, criterion)
torch.save(model, output_path + "Deepfake_Model_with_CNN.pt")

running epoch: 1


 33%|███▎      | 354/1081 [2:01:10<4:08:51, 20.54s/it]


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(221)
plt.plot(train_losses_his, 'b', label = 'training loss')
plt.plot(valid_losses_his, 'r', label = 'validation loss')
plt.title("ResNet Loss")
plt.legend(loc='upper left')
plt.subplot(222)
plt.plot(train_acc_his, 'b', label = 'trainingaccuracy')
plt.plot(valid_acc_his, 'r', label = 'validation accuracy')
plt.title("ResNet Accuracy")
plt.legend(loc = 'upper left')
plt.show()